In [1]:
import pandas as pd
import numpy as np

## 1. A Multiply Indexed Series

#### 1.1 The bad way

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [4]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

#### 1.2 The Better Way: Pandas MultiIndex

In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

Notice that the **MultiIndex** contains multiple levels of **indexing–in** this case, **the state names** and **the years**, as well as **multiple labels** for each **data point** which encode these levels.

In [6]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [12]:
pop['California', 2000]

33871648

Now to access **all data** for which **the second index** is 2010, we can simply use the **Pandas slicing notation**:

In [13]:
pop[:, 2000]

California    33871648
New York      18976457
Texas         20851820
dtype: int64

#### 2.3 MultiIndex as extra dimension

The **unstack()** method will quickly **convert a multiply indexed Series** into a **conventionally indexed DataFrame**:

In [14]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Naturally, **the stack()** method provides the **opposite operation**:

In [15]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Seeing this, you might wonder why would we would bother with **hierarchical indexing** at all. The reason is **simple**: just as we were able to use **multi-indexing** to **represent two-dimensional data** within a **one-dimensional Series**, we can also use it to represent data of **three or more dimensions** in **a Series or DataFrame**. 

Each **extra level** in a **multi-index** represents an **extra dimension of data**; taking advantage of this property gives us much more **flexibility** in the types of data we can represent. 

Concretely, we might want to add another **column of demographic data** for each state at each year (say, population under 18) ; with a MultiIndex this is as easy as adding another column to the DataFrame:

In [16]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [18]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18

California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64

In [20]:
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


## 2. Methods of MultiIndex Creation

In [21]:
# most straight way
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.897704  0.658060
  2  0.917334  0.083940
b 1  0.568813  0.984198
  2  0.008457  0.341514

The work of creating the **MultiIndex** is done in the background.

Similarly, if you pass a **dictionary** with **appropriate tuples** as **keys**, Pandas will automatically recognize this and use a MultiIndex by default:

In [22]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

**Nevertheless**, it is sometimes **useful to explicitly** create a MultiIndex; we'll see a couple of these methods here.

## 3. Explicit MultiIndex constructors